In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy
import os
import os.path 
import matplotlib.pyplot as plt
import matplotlib.colors
import copy
import errno
import glob
import time
import calendar
import json
import pickle
import random
import netCDF4
from numpy import savez_compressed
from optparse import OptionParser
from scipy.interpolate import (UnivariateSpline, RectBivariateSpline, RegularGridInterpolator)
from sklearn.metrics import confusion_matrix

In [3]:
import sys
sys.path.append('/data1/fog/Hamid/FogNet/') 
import src
from src import utils, FogNet, FogNetConfig, cnn_evaluate, xai_engine

## Channel

In [5]:
import pandas as pd
import numpy
import os
import os.path 
import matplotlib.pyplot as plt
import matplotlib.colors
import copy
import errno
import glob
import time
import calendar
import json
import pickle
import random
import netCDF4
import copy
from numpy import savez_compressed
from optparse import OptionParser
from scipy.interpolate import (UnivariateSpline, RectBivariateSpline, RegularGridInterpolator)
from sklearn.metrics import confusion_matrix
import statistics
import tensorflow
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model   
from tensorflow.keras.layers import Add, add, concatenate, Reshape, BatchNormalization, Input, Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, Conv3D, Activation, MaxPooling2D, MaxPooling3D, AveragePooling3D, ReLU, GlobalAveragePooling3D, multiply
 
from tensorflow.keras import regularizers 
from tensorflow.keras import optimizers 
from tensorflow.keras.optimizers import Adam, SGD 
from tensorflow.keras.callbacks import ModelCheckpoint


from scipy.io import loadmat
from tensorflow.keras.callbacks import EarlyStopping

DEFAULT_IMAGE_DIR_NAME = ('/data1/fog-data/fog-maps/')
DEFAULT_TARGET_DIR_NAME = ('/data1/fog/fognn/Dataset/TARGET/')

DEFAULT_CUBES_24_DIR_NAME = ('/data1/fog/fognn/Dataset/24HOURS/INPUT/')
DEFAULT_TARGET_24_DIR_NAME = ('/data1/fog/fognn/Dataset/24HOURS/TARGET/')

DEFAULT_LINE_COLOUR = numpy.array([228, 26, 28], dtype=float) / 255
DEFAULT_LINE_WIDTH = 3
DEFAULT_RANDOM_LINE_COLOUR = numpy.full(3, 152. / 255)
DEFAULT_RANDOM_LINE_WIDTH = 2

LEVELS_FOR_CONTOURS = numpy.linspace(0, 1, num=11, dtype=float)

FIGURE_WIDTH_INCHES = 10
FIGURE_HEIGHT_INCHES = 10

FONT_SIZE = 20
plt.rc('font', size=FONT_SIZE)
plt.rc('axes', titlesize=FONT_SIZE)
plt.rc('axes', labelsize=FONT_SIZE)
plt.rc('xtick', labelsize=FONT_SIZE)
plt.rc('ytick', labelsize=FONT_SIZE)
plt.rc('legend', fontsize=FONT_SIZE)
plt.rc('figure', titlesize=FONT_SIZE)

strategy = tensorflow.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

############################
# Setup input data rasters #
############################
# Generate data file paths
trainYearIdxs = [4, 5, 6, 7, 8]
valYearIdxs   = [0, 1, 2, 3]
testYearIdxs  = [9, 10, 11]

horizons = [6, 12, 24]
allYears = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

#strategy = tensorflow.distribute.experimental.MultiWorkerMirroredStrategy()
#print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
#with strategy.scope(): 
    
nam_G1_template = "NETCDF_NAM_CUBE_{year}_PhG1_{horizon}.npz"
nam_G1_names = [nam_G1_template.format(year=year, horizon=horizons[2]) for year in allYears]

nam_G2_template = "NETCDF_NAM_CUBE_{year}_PhG2_{horizon}.npz"
nam_G2_names = [nam_G2_template.format(year=year, horizon=horizons[2]) for year in allYears]

nam_G3_template = "NETCDF_NAM_CUBE_{year}_PhG3_{horizon}.npz"
nam_G3_names = [nam_G3_template.format(year=year, horizon=horizons[2]) for year in allYears]

nam_G4_template = "NETCDF_NAM_CUBE_{year}_PhG4_{horizon}.npz"
nam_G4_names = [nam_G4_template.format(year=year, horizon=horizons[2]) for year in allYears]

mixed_file_template = "NETCDF_MIXED_CUBE_{year}_{horizon}.npz"
mixed_file_names = [mixed_file_template.format(year=year, horizon=horizons[2]) for year in allYears]

mur_file_template = "NETCDF_SST_CUBE_{year}.npz"
mur_file_names = [mur_file_template.format(year=year) for year in allYears]

targets_file_template = "target{year}_{horizon}.csv"
targets_file_names = [targets_file_template.format(year=year, horizon=horizons[2]) for year in allYears]


    # Read data cubes
training_list   = utils.load_Cat_cube_data(nam_G1_names,
    nam_G2_names, nam_G3_names, nam_G4_names, mixed_file_names, mur_file_names, DEFAULT_CUBES_24_DIR_NAME, trainYearIdxs)

validation_list = utils.load_Cat_cube_data(nam_G1_names,
    nam_G2_names, nam_G3_names, nam_G4_names, mixed_file_names, mur_file_names, DEFAULT_CUBES_24_DIR_NAME, valYearIdxs)

testing_list    = utils.load_Cat_cube_data(nam_G1_names,
    nam_G2_names, nam_G3_names, nam_G4_names, mixed_file_names, mur_file_names, DEFAULT_CUBES_24_DIR_NAME, testYearIdxs)

target_class = utils.targets(
    targets_file_names, trainYearIdxs, valYearIdxs, testYearIdxs,
    DEFAULT_TARGET_24_DIR_NAME,
    0, # priority_calss: the last integer value is the class of target to predict: 0: is < 1600; 1: < 3200 and 2: < 6400
)
target_list = target_class.binary_target()

# Separate into train, test, validation
Training_targets = target_list[0]
print('training target shape:', Training_targets.shape)
ytrain = target_list[1]
print('training categorical target shape:', ytrain.shape)
Validation_targets = target_list[2]
print('validation target shape:', Validation_targets.shape)
yvalid = target_list[3]
print('validation categorical target shape:', yvalid.shape)
Testing_targets = target_list[4]
print('testing target shape:', Testing_targets.shape)
ytest = target_list[5]
print('testing categorical target shape:', ytest.shape)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Number of devices: 4
NAM_G1 input size:  (5460, 32, 32, 108, 1)
NAM_G2 input size:  (5460, 32, 32, 96, 1)
NAM_G3 input size:  (5460, 32, 32, 108, 1)
NAM_G4 input size:  (5460, 32, 32, 60, 1)
MIXED input size:  (5460, 32, 32, 12, 1)
MUR input size:  (5460, 384, 384, 1, 1)
NAM_G1 input size:  (3328, 32, 32, 108, 1)
NAM_G2 input size:  (3328, 32, 32, 96, 1)
NAM_G3 input size:  (3328, 32, 32, 108, 1)
NAM_G4 input size:  (3328, 32, 32, 60, 1)
MIXED input size:  (3328, 32, 32, 12, 1)
MUR input size:  (3328, 384, 384, 1, 1)
NAM_G1 input size:  (2228, 32, 32, 108, 1)
NAM_G2 input size:  (2228, 32, 32, 96, 1)
NAM_G3 input size:  (2228, 32, 32, 108, 1)
NAM_G4 input size:  (2228, 32, 32, 60, 1)
MIXED input size:  (2228, 32, 32, 12, 1)
MUR input size:  (222

In [6]:

#with strategy.scope():
    # Initialize
learningRate = 0.0009 # hyperparameters[key][0] 
wd           = 0.001  # hyperparameters[key][1] 
filters      = 24     # hyperparameters[key][2] 
dropout      = 0.3    # hyperparameters[key][3] 


cnn_file_name = '/data1/fog/FogNet/trained_model/single_gpu_weights.h5'
#cnn_file_name = '/data1/fog/fognn/FogNet/trained_model/weights.h5'
input_nam_shape       = Input((32, 32, 288, 1))
input_mur_shape       = Input((384, 384, 1, 1)) 
input_nam_G1_24_shape = Input((32, 32, 108, 1))
input_nam_G2_24_shape = Input((32, 32, 96, 1))
input_nam_G3_24_shape = Input((32, 32, 108, 1))
input_nam_G4_24_shape = Input((32, 32, 60, 1))
input_mixed_24_shape  = Input((32, 32, 12, 1))


C  = FogNet.FogNet(input_nam_G1_24_shape, 
               input_nam_G2_24_shape, 
               input_nam_G3_24_shape, 
               input_nam_G4_24_shape, 
               input_mixed_24_shape, 
               input_mur_shape, filters, dropout, 2)

cnn_model_object = C.BuildModel()
#cnn_model_object = multi_gpu_model(cnn_model_object, gpus=4)
#model.summary() 

cnn_model_object.load_weights(cnn_file_name)  

cnn_model_object.compile(optimizer=Adam(lr=learningRate, decay=wd),
      loss='categorical_crossentropy',
      metrics=['accuracy'])


In [7]:
def copy_group_names(gnames):
    out = []
    times = ['T1', 'T2', 'T3', 'T4']
    for name in gnames: 
        for t in times: 
            out.append(name +'_' + t)
            
    return out 
        

In [ ]:
df = pd.DataFrame()

mean_hss, mean_pss, mean_css = [], [], []
std_hss, std_pss, std_css = [], [], []
fnames, gnames = [], []

#n_groups   = len(input_data)

g = 5

if g ==0:
    GNames   = utils.NETCDF_PREDICTOR_NAMES['Physical_G1']
elif g ==1:
    GNames = utils.NETCDF_PREDICTOR_NAMES['Physical_G2']
elif g ==2:
    GNames = utils.NETCDF_PREDICTOR_NAMES['Physical_G3']
elif g ==3:
    GNames = utils.NETCDF_PREDICTOR_NAMES['Physical_G4']
elif g ==4:
    GNames = utils.NETCDF_MIXED_NAMES
elif g ==5:
    GNames = utils.NETCDF_MUR_NAMES  

GNames_t = copy_group_names(GNames)
n_features = testing_list[g].shape[3]


random_state=42
strategy = tensorflow.distribute.experimental.MultiWorkerMirroredStrategy()
#print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
with strategy.scope(): 
    for f in range(0, 4):

        this_hss, this_pss, this_css = [], [], []

        #for i in range(2):
            #print(f"Iteration {i}: group {g}, feature name {GNames[f]}!")

        input_data_copy    = copy.deepcopy(testing_list)
        numpy.random.seed(random_state)
        permuted_map       = numpy.random.permutation(input_data_copy[g][:,:,:,f,:]) 
        input_data_copy[g][:,:,:,f,:] = permuted_map

        y_testing_cat_prob = cnn_model_object.predict(input_data_copy, batch_size = 4) 
        metric_list        = cnn_evaluate.test_eval(Testing_targets, y_testing_cat_prob, threshold = 0.193)

        this_pss.append(metric_list[8])
        this_hss.append(metric_list[9])
        this_css.append(metric_list[11])

        feature_name   = GNames_t[f]
        fnames.append(feature_name)
        gnames.append(g)

        mean_pss.append(statistics.mean(this_pss))
        std_pss.append(statistics.pstdev(this_pss))

        mean_hss.append(statistics.mean(this_hss))
        std_hss.append(statistics.pstdev(this_hss))

        mean_css.append(statistics.mean(this_css))
        std_css.append(statistics.pstdev(this_css)) 
        
        print(f"{GNames_t[f]}| PSS = {statistics.mean(this_pss):.2f}, {statistics.pstdev(this_pss):.2f}| HSS = {statistics.mean(this_hss):.2f}, {statistics.pstdev(this_hss):.2f}| CSS = {statistics.mean(this_css):.2f}, {statistics.pstdev(this_css):.2f}")

INFO:tensorflow:Using MirroredStrategy with devices ('/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3')
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3'), communication = CommunicationImplementation.AUTO
analysed_sst_T1| PSS = 0.54, 0.00| HSS = 0.52, 0.00| CSS = 0.51, 0.00


In [9]:
GNames = utils.NETCDF_PREDICTOR_NAMES['Physical_G4']
len(GNames)

15

## FogNet (24 Hr. 1600m) based on PSS, HSS, and CSS: 

In [4]:
Training_targets  = pd.read_csv(os.path.join('/data1/fog/Hamid/FogNet/trained_model/VIS_True_TRAIN.csv')) 
Training_targets  = Training_targets[['CAT1600']].to_numpy().ravel()
Testing_targets      = pd.read_csv(os.path.join('/data1/fog/Hamid/FogNet/trained_model/VIS_True_TEST.csv'))
Testing_targets      = Testing_targets[['CAT1600']].to_numpy().ravel()

In [5]:
fognet_training_cat_prob  = pd.read_csv(os.path.join('/data1/fog/Hamid/FogNet/trained_model/VIS_Prob_TRAIN.csv'))
fognet_training_cat_prob  = fognet_training_cat_prob[['C0_Prob', 'C1_Prob']].to_numpy()
fognet_test_cat_prob      = pd.read_csv(os.path.join('/data1/fog/Hamid/FogNet/trained_model/VIS_Prob_TEST.csv'))
fognet_test_cat_prob      = fognet_test_cat_prob[['C0_Prob', 'C1_Prob']].to_numpy()

In [6]:
hss_training_threshold, hss_accuray_list, hss_Res241600  = cnn_evaluate.skilled_metrics(Training_targets, fognet_training_cat_prob, 'HSS')
hss_testing_accuracy                      = cnn_evaluate.test_eval(Testing_targets, fognet_test_cat_prob, hss_training_threshold)
pss_training_threshold, pss_accuray_list, pss_Res241600  = cnn_evaluate.skilled_metrics(Training_targets, fognet_training_cat_prob, 'PSS')
pss_testing_accuracy                      = cnn_evaluate.test_eval(Testing_targets, fognet_test_cat_prob, pss_training_threshold)
css_training_threshold, css_accuray_list, css_Res241600  = cnn_evaluate.skilled_metrics(Training_targets, fognet_training_cat_prob, 'CSS')
css_testing_accuracy                      = cnn_evaluate.test_eval(Testing_targets, fognet_test_cat_prob, css_training_threshold)

In [7]:
print(f"------  HSS  ------  PSS  ------  CSS")
print(f"POD:   {hss_testing_accuracy[4]:.2f}         {pss_testing_accuracy[4]:.2f}         {css_testing_accuracy[4]:.2f}")
print(f"F:     {hss_testing_accuracy[5]:.2f}         {pss_testing_accuracy[5]:.2f}         {css_testing_accuracy[5]:.2f}")
print(f"FAR:   {hss_testing_accuracy[6]:.2f}         {pss_testing_accuracy[6]:.2f}         {css_testing_accuracy[6]:.2f}")
print(f"CSI:   {hss_testing_accuracy[7]:.2f}         {pss_testing_accuracy[7]:.2f}         {css_testing_accuracy[7]:.2f}")
print(f"PSS:   {hss_testing_accuracy[8]:.2f}         {pss_testing_accuracy[8]:.2f}         {css_testing_accuracy[8]:.2f}")
print(f"HSS:   {hss_testing_accuracy[9]:.2f}         {pss_testing_accuracy[9]:.2f}         {css_testing_accuracy[9]:.2f}")
print(f"ORSS:  {hss_testing_accuracy[10]:.2f}         {pss_testing_accuracy[10]:.2f}         {css_testing_accuracy[10]:.2f}")
print(f"CSS:   {hss_testing_accuracy[11]:.2f}         {pss_testing_accuracy[11]:.2f}         {css_testing_accuracy[11]:.2f}")

------  HSS  ------  PSS  ------  CSS
POD:   0.55         0.58         0.28
F:     0.02         0.03         0.00
FAR:   0.49         0.61         0.32
CSI:   0.36         0.31         0.25
PSS:   0.54         0.55         0.28
HSS:   0.52         0.45         0.39
ORSS:  0.97         0.96         0.98
CSS:   0.50         0.38         0.66


### Permutation 

#### Based on HSS

In [8]:
pfi_df = pd.read_csv('./PFIGW.csv')
permut_hss_results = pfi_df['HSS_Mean']

permutation_hss_results = [(0.52 - permut_hss_results[i]) for i in range(5)]
print("Permutation HSS based results:")
print("------  Score")
print(f"G1:    {permutation_hss_results[0] * 100:.2f}")
print(f"G2:    {permutation_hss_results[1] * 100:.2f}")
print(f"G3:    {permutation_hss_results[2] * 100:.2f}")
print(f"G4:    {permutation_hss_results[3] * 100:.2f}")
print(f"G5:    {permutation_hss_results[4] * 100:.2f}")

Permutation HSS based results:
------  Score
G1:    1.46
G2:    -0.14
G3:    0.67
G4:    -0.98
G5:    2.23


#### Based on PSS 

In [13]:
pfi_df = pd.read_csv('./PFIGW.csv')
permut_pss_results = pfi_df['PSS_Mean']

permutation_pss_results = [(0.55 - permut_pss_results[i]) for i in range(5)]
print("Permutation PSS based results:")
print("------  Score")
print(f"G1:    {permutation_pss_results[0] * 100:.2f}")
print(f"G2:    {permutation_pss_results[1] * 100:.2f}")
print(f"G3:    {permutation_pss_results[2] * 100:.2f}")
print(f"G4:    {permutation_pss_results[3] * 100:.2f}")
print(f"G5:    {permutation_pss_results[4] * 100:.2f}")

Permutation PSS based results:
------  Score
G1:    1.53
G2:    1.35
G3:    1.44
G4:    1.26
G5:    1.63


### Group Hold Hout

In [27]:
onehout_dir = '/data1/fog/Hamid/FogNet/trained_model/xai_trained_models/onehout/'
onehout_folder_list = sorted(os.listdir(onehout_dir))

#### Maximize Based on HSS and return HSS

In [32]:
onehout_hss_results, onehout_pss_results = [], []

for i in range(len(onehout_folder_list)):
    y_training_cat_prob  = pd.read_csv(os.path.join(onehout_dir + onehout_folder_list[i] + '/VIS_Prob_TRAIN.csv'))
    y_training_cat_prob  = y_training_cat_prob[['C0_Prob', 'C1_Prob']].to_numpy()
    y_test_cat_prob      = pd.read_csv(os.path.join(onehout_dir + onehout_folder_list[i] + '/VIS_Prob_TEST.csv'))
    y_test_cat_prob      = y_test_cat_prob[['C0_Prob', 'C1_Prob']].to_numpy()
    
    training_threshold_hss, _, _   = cnn_evaluate.skilled_metrics(Training_targets, y_training_cat_prob, 'HSS')
    training_threshold_pss, _, _   = cnn_evaluate.skilled_metrics(Training_targets, y_training_cat_prob, 'PSS')
    
    testing_accuracy_basedon_hss =  cnn_evaluate.test_eval(Testing_targets, y_test_cat_prob, training_threshold_hss)
    testing_accuracy_basedon_pss =  cnn_evaluate.test_eval(Testing_targets, y_test_cat_prob, training_threshold_pss)


    onehout_hss_results.append(testing_accuracy_basedon_hss) 
    onehout_pss_results.append(testing_accuracy_basedon_pss)


#### Scores based on HSS and Maximize based on HSS

In [35]:
onegout_only_hss_basedon_hss = [(0.52 - onehout_hss_results[i][9]) for i in range(5)]
print("OneGHoldout HSS based on HSS maximized results:")
print("------ Score")
print(f"G1:    {onegout_only_hss_basedon_hss[0] * 100:.2f}")
print(f"G2:    {onegout_only_hss_basedon_hss[1] * 100:.2f}")
print(f"G3:    {onegout_only_hss_basedon_hss[2] * 100:.2f}")
print(f"G4:    {onegout_only_hss_basedon_hss[3] * 100:.2f}")
print(f"G5:    {onegout_only_hss_basedon_hss[4] * 100:.2f}")

OneGHoldout HSS based on HSS maximized results:
------ Score
G1:    21.44
G2:    37.32
G3:    42.10
G4:    41.70
G5:    35.58


#### Scores based on PSS and Maximize based on HSS 

In [38]:
onegout_only_pss_basedon_hss = [(0.54 - onehout_hss_results[i][11]) for i in range(5)]
print("OneGHoldout PSS based on HSS maximized results:")
print("------ Score")
print(f"G1:    {onegout_only_pss_basedon_hss[0] * 100:.2f}")
print(f"G2:    {onegout_only_pss_basedon_hss[1] * 100:.2f}")
print(f"G3:    {onegout_only_pss_basedon_hss[2] * 100:.2f}")
print(f"G4:    {onegout_only_pss_basedon_hss[3] * 100:.2f}")
print(f"G5:    {onegout_only_pss_basedon_hss[4] * 100:.2f}")

OneGHoldout PSS based on HSS maximized results:
------ Score
G1:    26.55
G2:    23.39
G3:    34.08
G4:    -0.31
G5:    6.71


#### Scores based on PSS and Maximize based on PSS 

In [37]:
onegout_only_pss_basedon_pss = [(0.554324628592346 - onehout_pss_results[i][11]) for i in range(5)]
print("OneGHoldout PSS based on PSS maximized results:")
print("------ Score")
print(f"G1:    {onegout_only_pss_basedon_pss[0] * 100:.2f}")
print(f"G2:    {onegout_only_pss_basedon_pss[1] * 100:.2f}")
print(f"G3:    {onegout_only_pss_basedon_pss[2] * 100:.2f}")
print(f"G4:    {onegout_only_pss_basedon_pss[3] * 100:.2f}")
print(f"G5:    {onegout_only_pss_basedon_pss[4] * 100:.2f}")

OneGHoldout PSS based on PSS maximized results:
------ Score
G1:    30.39
G2:    39.95
G3:    35.08
G4:    25.83
G5:    22.68


#### Scores based on HSS and Maximize based on PSS 

In [41]:
onegout_only_hss_basedon_pss = [(0.45 - onehout_pss_results[i][9]) for i in range(5)]
print("OneGHoldout HSS based on PSS maximized results:")
print("------ Score")
print(f"G1:    {onegout_only_hss_basedon_pss[0] * 100:.2f}")
print(f"G2:    {onegout_only_hss_basedon_pss[1] * 100:.2f}")
print(f"G3:    {onegout_only_hss_basedon_pss[2] * 100:.2f}")
print(f"G4:    {onegout_only_hss_basedon_pss[3] * 100:.2f}")
print(f"G5:    {onegout_only_hss_basedon_pss[4] * 100:.2f}")

OneGHoldout HSS based on PSS maximized results:
------ Score
G1:    16.02
G2:    23.96
G3:    35.10
G4:    13.88
G5:    14.95


### LossSHAP

In [103]:
def LossSHAP_Scores(df, hss):
    
    Weights = [0.2, 0.2, 0.2, 0.2, 0.2, 
               0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 
               0.0333333,  0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 
               0.05, 0.05, 0.05, 0.05, 0.05,
               0.2] 

    G1_idx = [0, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20, 25, 26, 27, 28, 30]
    G1_weights = [Weights[index] for index in G1_idx]   
    G1_Sub = [(df.iloc[0]['HSS_Mean'] - hss), 
              (df.iloc[5]['HSS_Mean'] - df.iloc[1]['HSS_Mean']), 
              (df.iloc[6]['HSS_Mean'] - df.iloc[2]['HSS_Mean']), 
              (df.iloc[7]['HSS_Mean'] - df.iloc[3]['HSS_Mean']), 
              (df.iloc[8]['HSS_Mean'] - df.iloc[4]['HSS_Mean']), 
              (df.iloc[15]['HSS_Mean'] - df.iloc[9]['HSS_Mean']),  
              (df.iloc[16]['HSS_Mean'] - df.iloc[10]['HSS_Mean']), 
              (df.iloc[17]['HSS_Mean'] - df.iloc[11]['HSS_Mean']), 
              (df.iloc[18]['HSS_Mean'] - df.iloc[12]['HSS_Mean']), 
              (df.iloc[19]['HSS_Mean'] - df.iloc[13]['HSS_Mean']), 
              (df.iloc[20]['HSS_Mean'] - df.iloc[14]['HSS_Mean']), 
              (df.iloc[25]['HSS_Mean'] - df.iloc[21]['HSS_Mean']),
              (df.iloc[26]['HSS_Mean'] - df.iloc[22]['HSS_Mean']), 
              (df.iloc[27]['HSS_Mean'] - df.iloc[23]['HSS_Mean']), 
              (df.iloc[28]['HSS_Mean'] - df.iloc[24]['HSS_Mean']), 
              (hss - df.iloc[29]['HSS_Mean'])]
    G1_score = sum([a*b for a, b in zip(G1_weights, G1_Sub)])
    
    G2_idx = [1, 5, 9, 10, 11, 15, 16, 17, 21, 22, 23, 25, 26, 27, 29, 30]
    G2_weights = [Weights[index] for index in G2_idx]
    G2_Sub = [(df.iloc[1]['HSS_Mean'] - hss), 
              (df.iloc[5]['HSS_Mean'] - df.iloc[0]['HSS_Mean']), 
              (df.iloc[9]['HSS_Mean'] - df.iloc[2]['HSS_Mean']), 
              (df.iloc[10]['HSS_Mean'] - df.iloc[3]['HSS_Mean']), 
              (df.iloc[11]['HSS_Mean'] - df.iloc[4]['HSS_Mean']), 
              (df.iloc[15]['HSS_Mean'] - df.iloc[6]['HSS_Mean']),  
              (df.iloc[16]['HSS_Mean'] - df.iloc[7]['HSS_Mean']), 
              (df.iloc[17]['HSS_Mean'] - df.iloc[8]['HSS_Mean']), 
              (df.iloc[21]['HSS_Mean'] - df.iloc[12]['HSS_Mean']), 
              (df.iloc[22]['HSS_Mean'] - df.iloc[13]['HSS_Mean']), 
              (df.iloc[23]['HSS_Mean'] - df.iloc[14]['HSS_Mean']), 
              (df.iloc[25]['HSS_Mean'] - df.iloc[18]['HSS_Mean']),
              (df.iloc[26]['HSS_Mean'] - df.iloc[19]['HSS_Mean']), 
              (df.iloc[27]['HSS_Mean'] - df.iloc[20]['HSS_Mean']), 
              (df.iloc[29]['HSS_Mean'] - df.iloc[24]['HSS_Mean']), 
              (hss - df.iloc[28]['HSS_Mean'])]
    G2_score = sum([a*b for a, b in zip(G2_weights, G2_Sub)])
 
    G3_idx = [2, 6, 9, 12, 13, 15, 18, 19, 21, 22, 24, 25, 26, 28, 29, 30]
    G3_weights = [Weights[index] for index in G3_idx]
    G3_Sub = [(df.iloc[2]['HSS_Mean'] - hss), 
              (df.iloc[6]['HSS_Mean'] - df.iloc[0]['HSS_Mean']), 
              (df.iloc[9]['HSS_Mean'] - df.iloc[1]['HSS_Mean']), 
              (df.iloc[12]['HSS_Mean'] - df.iloc[3]['HSS_Mean']), 
              (df.iloc[13]['HSS_Mean'] - df.iloc[4]['HSS_Mean']), 
              (df.iloc[15]['HSS_Mean'] - df.iloc[5]['HSS_Mean']),  
              (df.iloc[18]['HSS_Mean'] - df.iloc[7]['HSS_Mean']), 
              (df.iloc[19]['HSS_Mean'] - df.iloc[8]['HSS_Mean']), 
              (df.iloc[21]['HSS_Mean'] - df.iloc[10]['HSS_Mean']), 
              (df.iloc[22]['HSS_Mean'] - df.iloc[11]['HSS_Mean']), 
              (df.iloc[24]['HSS_Mean'] - df.iloc[14]['HSS_Mean']), 
              (df.iloc[25]['HSS_Mean'] - df.iloc[16]['HSS_Mean']),
              (df.iloc[26]['HSS_Mean'] - df.iloc[17]['HSS_Mean']), 
              (df.iloc[28]['HSS_Mean'] - df.iloc[20]['HSS_Mean']), 
              (df.iloc[29]['HSS_Mean'] - df.iloc[23]['HSS_Mean']), 
              (hss - df.iloc[27]['HSS_Mean'])]
    G3_score = sum([a*b for a, b in zip(G3_weights, G3_Sub)])

    G4_idx = [3, 7, 10, 12, 14, 16, 18, 20, 21, 23, 24, 26, 27, 28, 29, 30]
    G4_weights = [Weights[index] for index in G4_idx]
    G4_Sub = [(df.iloc[3]['HSS_Mean'] - hss), 
              (df.iloc[7]['HSS_Mean'] - df.iloc[0]['HSS_Mean']), 
              (df.iloc[10]['HSS_Mean'] - df.iloc[1]['HSS_Mean']), 
              (df.iloc[12]['HSS_Mean'] - df.iloc[2]['HSS_Mean']), 
              (df.iloc[14]['HSS_Mean'] - df.iloc[4]['HSS_Mean']), 
              (df.iloc[16]['HSS_Mean'] - df.iloc[5]['HSS_Mean']),  
              (df.iloc[18]['HSS_Mean'] - df.iloc[6]['HSS_Mean']), 
              (df.iloc[20]['HSS_Mean'] - df.iloc[8]['HSS_Mean']), 
              (df.iloc[21]['HSS_Mean'] - df.iloc[9]['HSS_Mean']), 
              (df.iloc[23]['HSS_Mean'] - df.iloc[11]['HSS_Mean']), 
              (df.iloc[24]['HSS_Mean'] - df.iloc[13]['HSS_Mean']), 
              (df.iloc[25]['HSS_Mean'] - df.iloc[15]['HSS_Mean']),
              (df.iloc[27]['HSS_Mean'] - df.iloc[17]['HSS_Mean']), 
              (df.iloc[28]['HSS_Mean'] - df.iloc[19]['HSS_Mean']), 
              (df.iloc[29]['HSS_Mean'] - df.iloc[22]['HSS_Mean']), 
              (hss - df.iloc[26]['HSS_Mean'])]
    G4_score = sum([a*b for a, b in zip(G4_weights, G4_Sub)])

    G5_idx     = [4, 8, 11, 13, 14, 17, 19, 20, 22, 23, 24, 26, 27, 28, 29, 30] 
    G5_weights = [Weights[index] for index in G5_idx]
    G5_Sub = [(df.iloc[4]['HSS_Mean'] - hss), 
              (df.iloc[8]['HSS_Mean'] - df.iloc[0]['HSS_Mean']), 
              (df.iloc[11]['HSS_Mean'] - df.iloc[1]['HSS_Mean']), 
              (df.iloc[13]['HSS_Mean'] - df.iloc[2]['HSS_Mean']), 
              (df.iloc[14]['HSS_Mean'] - df.iloc[3]['HSS_Mean']), 
              (df.iloc[17]['HSS_Mean'] - df.iloc[5]['HSS_Mean']),  
              (df.iloc[19]['HSS_Mean'] - df.iloc[6]['HSS_Mean']), 
              (df.iloc[20]['HSS_Mean'] - df.iloc[7]['HSS_Mean']), 
              (df.iloc[22]['HSS_Mean'] - df.iloc[9]['HSS_Mean']), 
              (df.iloc[23]['HSS_Mean'] - df.iloc[10]['HSS_Mean']), 
              (df.iloc[24]['HSS_Mean'] - df.iloc[12]['HSS_Mean']), 
              (df.iloc[26]['HSS_Mean'] - df.iloc[15]['HSS_Mean']),
              (df.iloc[27]['HSS_Mean'] - df.iloc[16]['HSS_Mean']), 
              (df.iloc[28]['HSS_Mean'] - df.iloc[18]['HSS_Mean']), 
              (df.iloc[29]['HSS_Mean'] - df.iloc[21]['HSS_Mean']), 
              (hss - df.iloc[25]['HSS_Mean'])]

    G5_score = sum([a*b for a, b in zip(G5_weights, G5_Sub)])
    
    
    return G1_score, G2_score, G3_score, G4_score, G5_score

In [104]:
LossSHAP_df = pd.read_csv('./LossSHAP.csv')

G1_score, G2_score, G3_score, G4_score, G5_score = LossSHAP_Scores(LossSHAP_df, 0.52)

In [105]:
print("LossSHAP HSS based results:")
print("------ Score")
print(f"G1:    {G1_score * 100:.2f}")
print(f"G2:    {G2_score * 100:.2f}")
print(f"G3:    {G3_score * 100:.2f}")
print(f"G4:    {G4_score * 100:.2f}")
print(f"G5:    {G5_score * 100:.2f}")

LossSHAP HSS based results:
------ Score
G1:    -9.39
G2:    -4.19
G3:    -6.33
G4:    14.16
G5:    5.75
